In [1]:
# We import Libraries 
import pandas as pd 
import numpy as np
import json 

In [2]:
# We load the raw data to check the total number of records before cleaning
with open('../data/raw_credit_applications.json', 'r') as f:
    data = json.load(f)

print(f"Total raw records: {len(data)}")

Total raw records: 502


In [15]:
# We can use json_normalize to flatten the nested JSON structure into a DataFrame
df = pd.json_normalize(data)
df.head(100)

,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,41,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN
3,app_024,"[{'category': 'Fitness', 'amount': 575}]",NaN,Thomas Lee,thomas.lee6@protonmail.com,194-35-1833,192.168.175.67,Male,1983-04-25,10077,...,70,0.35,0,True,NaN,NaN,4.3,34000.0,NaN,NaN
4,app_184,"[{'category': 'Entertainment', 'amount': 463}]",2024-01-15T00:00:00Z,Brian Rodriguez,brian.rodriguez86@aol.com,480-41-2475,172.29.125.105,M,1999-05-21,10080,...,14,0.23,31763,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,app_323,"[{'category': 'Travel', 'amount': 133}, {'cate...",NaN,Michael Thompson,michael.thompson45@gmail.com,862-33-2796,172.26.169.208,Male,08/11/1981,10090,...,92,0.40,27746,True,NaN,NaN,3.9,80000.0,NaN,NaN
96,app_288,"[{'category': 'Shopping', 'amount': 434}]",NaN,Susan Young,susan.young22@mail.com,159-19-1765,192.168.114.250,Female,1962-11-05,90252,...,71,0.27,23054,True,NaN,NaN,5.5,67000.0,NaN,NaN
97,app_396,"[{'category': 'Shopping', 'amount': 698}, {'ca...",NaN,Katherine Harris,katherine.harris16@outlook.com,174-24-9597,10.14.75.95,Female,1976-01-28,10027,...,124,0.20,33232,True,NaN,NaN,5.4,40000.0,NaN,NaN
98,app_371,"[{'category': 'Rent', 'amount': 180}, {'catego...",NaN,Sarah Walker,sarah.walker95@mail.com,142-97-2132,10.57.113.229,Female,1995-02-23,90250,...,12,0.11,59032,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN


In [4]:
# We check for missing values in each column
df.isnull().sum().sort_values(ascending=False)

notes                               500
financials.annual_salary            497
loan_purpose                        452
processing_timestamp                440
decision.rejection_reason           292
decision.approved_amount            210
decision.interest_rate              210
financials.annual_income              5
applicant_info.ip_address             5
applicant_info.ssn                    5
applicant_info.date_of_birth          1
applicant_info.zip_code               1
applicant_info.gender                 1
spending_behavior                     0
financials.credit_history_months      0
financials.debt_to_income             0
financials.savings_balance            0
decision.loan_approved                0
applicant_info.email                  0
applicant_info.full_name              0
_id                                   0
dtype: int64

In [5]:
# We can drop notes column as it has a high percentage of missing values
df = df.drop(columns=["notes"])


In [6]:
# We investigate the duplicate records by ID (the primary key)
duplicate_count = df["_id"].duplicated().sum()
print(f"Duplicate ID count: {duplicate_count}")

Duplicate ID count: 2


In [7]:
## Data Quality Issue 1 - Duplicate Records
# we found 2 duplicate records based on the "_id" column. This is a critical issue because the "_id" field is supposed to be a unique identifier for each record.
# duplicates violate uniqueness constraints and compromise data integrity

duplicates = df[df["_id"].duplicated(keep=False)].sort_values("_id")

# we can see that some records have more missing values than others. We will keep the most complete record for each duplicate ID.
df["completeness_score"] = df.notnull().sum(axis=1)


df = df.sort_values(
    by=["_id", "completeness_score"],
    ascending=[True, False]
)

# we remove both the duplicate records and the completeness score column
df = df.drop_duplicates(subset="_id", keep="first")
df = df.drop(columns=["completeness_score"])

print("Remaining duplicate IDs:", df["_id"].duplicated().sum())

Remaining duplicate IDs: 0


In [14]:
## Data Quality Issue 2 – Column Inconsistency (annual_income vs annual_salary)
df["financials.income_unified"] = df["financials.annual_income"]

# we fill missing income with annual_salary
df.loc[df["financials.income_unified"].isnull(), "financials.income_unified"] = \
    df["financials.annual_salary"]

# Check how many records still have missing income after filling
missing_income_after = df["financials.income_unified"].isnull().sum()
print(f"Missing income records after filling: {missing_income_after}")

Missing income records after filling: 0


In [ ]:
## Data Quality Issue 3 – Categorical
# we standardize the gender column to ensure consistent values M = male  
def standardize_gender(df):
    df["applicant_info.gender"] = (
        df["applicant_info.gender"]
        .astype(str)
        .str.strip()
        .str.lower()
    )
   
    df["applicant_info.gender"] = df["applicant_info.gender"].replace({
        "m": "male",
        "male ": "male",
        "f": "female",
        "female ": "female"
    })
    
    df.loc[df["applicant_info.gender"] == "nan", "applicant_info.gender"] = np.nan
    
    return df

df = standardize_gender(df)

# we check the value counts for gender to verify the standardization
df["applicant_info.gender"].value_counts()

applicant_info.gender
female    251
male      248
            2
Name: count, dtype: int64